

# Modélisation du taux de réussite et de la mention

Ce notebook est consacré à la **modélisation du taux de réussite et du taux de mention au baccalauréat**. Les données exploitées proviennent du notebook exploratoire `01_exploratoires.ipynb`, où elles ont été nettoyées, enrichies et mises à disposition pour l’analyse statistique et la modélisation.

---

## 1. Données disponibles

Les informations collectées et traitées incluent :

### a) Lycées et leur géolocalisation
- Informations techniques sur les établissements :  
  - Nom du lycée  
  - Département et région  
  - Secteur (public/privé)  
- Effectifs des élèves présents au bac, selon les différentes filières  
- Taux de réussite et de mention par filière  
- Nombre de mentions détaillé à l’échelle agrégée  
- Indicateurs de **valeur ajoutée** pour le taux de réussite et le taux de mention  

### b) Bibliothèques à proximité
- Localisation géographique des bibliothèques autour des lycées  
- **Tranches exclusives de proximité** pour éviter la multicolinéarité et clarifier l’influence de la distance :  
  - `n_biblio_0_50` : nombre de bibliothèques très proches, situées **entre 0 et 50 m** du lycée  
  - `n_biblio_50_150` : nombre de bibliothèques **entre 50 et 150 m**, zone moyennement proche  
  - `n_biblio_150_300` : nombre de bibliothèques **entre 150 et 300 m**, zone à distance moyenne  
  - `n_biblio_300_500` : nombre de bibliothèques **entre 300 et 500 m**, zone éloignée  
  - `n_biblio_500_1000` : nombre de bibliothèques **entre 500 et 1000 m**, zone très éloignée  

> Cette approche permet de **capturer l’effet des bibliothèques selon leur distance** tout en gardant un modèle stable et interprétable. Chaque bibliothèque est comptée dans une seule tranche, évitant ainsi la double comptabilisation et la multicolinéarité.

### c) Indice de Position Sociale (IPS)
L’IPS renseigne sur la **composition sociale des élèves au sein de l’établissement** et s’inspire directement de la théorie bourdieusienne :  
- À partir de la PCS (catégorie socioprofessionnelle) des parents d’élèves, on déduit une **position sociale moyenne par lycée**  
- **Interprétation sociologique** : plus l’IPS est élevé, plus la position sociale moyenne des élèves est élevée, et plus ces derniers sont supposés mieux réussir au bac  

---

## 2. Variables d’intérêt

### a) Variables explicatives
- `ips` : indice de position sociale moyen du lycée  
- `d_lycee` : département du lycée  
- `s_lycee` : secteur du lycée (public/privé)  
- `existe_prepa` : présence de classes préparatoires dans le lycée (booléen)  
- Bibliothèques à proximité :  
  - `n_biblio_0_50` (très proche, 0-50 m)  
  - `n_biblio_50_150` (moyennement proche, 50-150 m)  
  - `n_biblio_150_300` (distance moyenne, 150-300 m)  
  - `n_biblio_300_500` (éloignée, 300-500 m)  
  - `n_biblio_500_1000` (très éloignée, 500-1000 m)  
- Autres variables contextuelles pertinentes (ex. filière dominante, effectifs totaux, etc.)  

### b) Variable cible
- `taux_reussite` : proportion d’élèves ayant réussi au baccalauréat dans le lycée  

---

## 3. Modèle statistique adopté

Pour modéliser le **taux de réussite**, nous utilisons une **régression linéaire log-niveau** :  
$$
\log(\text{taux\_reussite}) = 
\beta_0 + \beta_1 \cdot ips + \beta_2 \cdot d\_lycee + \beta_3 \cdot s\_lycee + \beta_4 \cdot existe\_prepa + \beta_5 \cdot n\_biblio\_0\_50 + \beta_6 \cdot n\_biblio\_50\_150 + \beta_7 \cdot n\_biblio\_150\_300 + \beta_8 \cdot n\_biblio\_300\_500 + \beta_9 \cdot n\_biblio\_500\_1000
$$



### Pourquoi une transformation logarithmique ?
- La **log-transformation de la variable cible** stabilise la variance et rend les résidus plus proches d’une distribution normale  
- Cela facilite l’interprétation des coefficients : un coefficient \(\beta_i\) mesure approximativement l’effet **en pourcentage** d’une variation d’une unité de la variable explicative sur le taux de réussite  

### Interprétation attendue
- `β1` (IPS) : impact de la position sociale moyenne sur le taux de réussite  
- `β4` (existe_prepa) : effet de la présence d’une classe préparatoire  
- `β5` à `β9` (bibliothèques) : influence de l’accès aux bibliothèques selon la distance sur la réussite  

---

## 4. Objectifs et perspectives

Ce modèle constitue une première étape pour comprendre **les déterminants sociaux et institutionnels de la réussite au bac**. Les objectifs incluent :  

- Identifier les variables les plus influentes sur le taux de réussite  
- Étudier l’effet différentiel des bibliothèques selon la distance  
- Servir de base pour des modèles plus avancés (interactions, variables catégorielles supplémentaires, analyses résiduelles)  


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_final = pd.read_csv("data/data_final.csv")

In [ ]:
data_final["n_biblio_0_50"] = data_final["n_biblio_50m"]
data_final["n_biblio_50_150"] = data_final["n_biblio_150m"] - data_final["n_biblio_50m"]
data_final["n_biblio_150_300"] = data_final["n_biblio_300m"] - data_final["n_biblio_150m"]
data_final["n_biblio_300_500"] = data_final["n_biblio_500m"] - data_final["n_biblio_300m"]
data_final["n_biblio_500_1000"] = data_final["n_biblio_1000m"] - data_final["n_biblio_500m"]

biblio_cols = ["n_biblio_0_50", "n_biblio_50_150", "n_biblio_150_300", "n_biblio_300_500", "n_biblio_500_1000"]

In [ ]:
X = data_final[["ips", "d_lycee", "s_lycee", "existe_prepa"] + biblio_cols]
y = data_final["taux_reussite"]
X = sm.add_constant(X)
y_log = np.log(y)

In [ ]:
model = sm.OLS(y_log, X).fit()
print(model.summary())